# train models and parameters tuning - RF

In [1]:
from os import fsdecode
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate

In [2]:
#read data from dataset (after categorical feature preprocessing and anomaly detection(IF))
# train model: aftcat,aftif, CV(Pipeline(MinMaxScaler, selector, model))
data_aftcat_aftif = pd.read_csv(r"./1-2 5388 traindata_aftcat_aftif.csv")
data_aftcat_aftif_y = pd.read_csv(r"./1-2 5388 traindata_aftcat_aftif_label.csv")

# predict model: aftcat, aftif, aftmm, aftfs
data_aftcat_aftif_aftmm_aftfs = pd.read_csv(r"./1-4 5388 traindata_aftcat_aftif_aftmm_aftfs.csv")
testdata_aftcat_aftmm_aftfs = pd.read_csv(r"./1-4 5388 testdata_aftcat_aftmm_aftfs.csv") 

In [3]:
data_aftcat_aftif

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,295,2584,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,312,839,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0,1225,2451,0,0,0,28,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0,293,296,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22989,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22990,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
22991,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
22992,0,516,4,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
data_aftcat_aftif_y

,Class
0,0
1,0
2,1
3,0
4,0
...,...
22989,0
22990,1
22991,1
22992,0


In [5]:
data_aftcat_aftif_aftmm_aftfs

,wrong_fragment,logged_in,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,...,service_time,service_urp_i,service_uucp,service_uucp_path,service_whois,flag_REJ,flag_RSTO,flag_RSTR,flag_S0,flag_SF
0,0.0,1.0,0.0,0.001961,0.003922,0.0,0.0,0.0,0.0,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.007843,0.011765,0.0,0.0,0.0,0.0,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.003922,0.005882,0.0,0.0,0.0,0.0,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.001961,0.000000,0.0,0.0,0.5,1.0,0.50,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22989,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1.0,1.0,1.00,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
22990,0.0,0.0,0.0,0.401961,0.023529,1.0,1.0,0.0,0.0,0.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
22991,0.0,0.0,0.0,0.237255,0.001961,1.0,1.0,0.0,0.0,0.02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
22992,0.0,0.0,0.0,0.527451,0.527451,0.0,0.0,0.0,0.0,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
testdata_aftcat_aftmm_aftfs

,wrong_fragment,logged_in,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,...,service_time,service_urp_i,service_uucp,service_uucp_path,service_whois,flag_REJ,flag_RSTO,flag_RSTR,flag_S0,flag_SF
0,0.0,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.468627,0.007843,1.0,1.0,0.0,0.0,0.02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.000000,1.000000,0.0,0.0,0.0,0.0,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.607843,0.607843,0.0,0.0,0.0,0.0,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22496,0.0,1.0,0.0,0.039216,0.039216,0.0,0.0,0.0,0.0,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
22497,0.0,1.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
22498,0.0,0.0,0.0,0.288235,0.033333,1.0,1.0,0.0,0.0,0.12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
22499,0.0,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [7]:
# function score_mean_std: calculate the average and std of scores
def score_mean_std(score_sf):
    index_a1 = []
    index_a2 = []

    for i in range(score_sf.shape[1]):
        index_a1.append(np.average(score_sf[score_sf.columns[i]]))
        index_a2.append(np.std(score_sf[score_sf.columns[i]]))

    index_a1 = pd.DataFrame(index_a1, index = score_sf.columns)
    index_a2 = pd.DataFrame(index_a2, index = score_sf.columns)
    #print(index_a1.T)
    #print(index_a2.T)
    re = pd.concat([index_a1.T,index_a2.T],axis=0)
    re.index = ["avg","std"]
    #print(re)
    return re

In [8]:
# -------------- train predictive model with pipeline: MinMaxScaler(), VarianceThreshold(), cross_validate()--------
def train_model(m, se, model, X, y):
    #print(model, "\n")
    pipe_steps = [('mm',m),('selector',se),('model',model)]
    id_pipeline = Pipeline(steps=pipe_steps)

    # evaluate the pipeline using the crossvalidation technique defined in cv

    # ------------------- score ------------------------------------
    scoring = {'f1', 'precision', 'accuracy',
            'recall', 'roc_auc'}

    score_sf = cross_validate(id_pipeline, X, y.values.ravel(),cv=10,scoring=scoring, n_jobs=-1)
    score_sf_re = score_mean_std(pd.DataFrame(score_sf))

    return score_sf,score_sf_re

In [9]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ Prediction  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
def pre_submission(model,X, y, X_test):
    print(model,"\n")
    model.fit(X, y.values.ravel())

    y_pre = model.predict(X_test).astype(int)
    print("sum:",y_pre.sum())

    pre = pd.DataFrame(columns=["ID","Class"])
    pre["ID"] = range(y_pre.shape[0])
    pre["Class"] = y_pre
    return pre

# Parameters tuning for Random Forest

#### Round #1 - default parameters

In [10]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
#model_rf = RandomForestClassifier(random_state = 90, min_samples_split=2,n_estimators=61,max_depth=24, max_features=27,min_samples_leaf=1, n_jobs=-1)
model_rf = RandomForestClassifier(random_state = 90)

In [11]:
# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)


RandomForestClassifier(random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_accuracy  test_roc_auc  test_precision  \
avg  2.721044    0.203421       0.998391      0.999986        0.998184   
std  0.684497    0.060314       0.000647      0.000010        0.001217   

     test_recall   test_f1  
avg     0.998455  0.998319  
std     0.000710  0.000675  


In [12]:
#  ----------------------------- RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_1.csv',index=None)

RandomForestClassifier(random_state=90) 

sum: 9819


#### Round #2 - n_estimators 

In [13]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#score_sf_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)

scorel = []
for i in range(1,200):
    model_rf = RandomForestClassifier(n_estimators=i,n_jobs=-1,random_state = 90)
    score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
    #print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
    scorel.append(np.array(score_sf_rf['test_f1']).mean())
    #print(np.array(score_sf_rf['test_f1']).mean())
print(max(scorel), scorel.index(max(scorel))+1)

RandomForestClassifier(n_estimators=1, n_jobs=-1, random_state=90) 

RandomForestClassifier(n_estimators=2, n_jobs=-1, random_state=90) 

RandomForestClassifier(n_estimators=3, n_jobs=-1, random_state=90) 

RandomForestClassifier(n_estimators=4, n_jobs=-1, random_state=90) 

RandomForestClassifier(n_estimators=5, n_jobs=-1, random_state=90) 

RandomForestClassifier(n_estimators=6, n_jobs=-1, random_state=90) 

RandomForestClassifier(n_estimators=7, n_jobs=-1, random_state=90) 

RandomForestClassifier(n_estimators=8, n_jobs=-1, random_state=90) 

RandomForestClassifier(n_estimators=9, n_jobs=-1, random_state=90) 

RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state=90) 

RandomForestClassifier(n_estimators=11, n_jobs=-1, random_state=90) 

RandomForestClassifier(n_estimators=12, n_jobs=-1, random_state=90) 

RandomForestClassifier(n_estimators=13, n_jobs=-1, random_state=90) 

RandomForestClassifier(n_estimators=14, n_jobs=-1, random_state=90) 

RandomForestClassifier(n_esti

In [ ]:
# 0.9985005563412249 167

In [56]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,n_jobs=-1)

# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(n_estimators=167, n_jobs=-1, random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_recall  test_precision  test_roc_auc  \
avg   4.07275    0.225083     0.998727        0.998276      0.999987   
std   0.67089    0.032121     0.000833        0.001245      0.000009   

     test_accuracy   test_f1  
avg       0.998565  0.998501  
std       0.000517  0.000539  


In [57]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_1.csv',index=None)

RandomForestClassifier(n_estimators=167, n_jobs=-1, random_state=90) 

sum: 9952


#### Round #3 - max_depth

In [62]:
param_grid = {'max_depth':np.arange(10,40,1)}
model_rf = RandomForestClassifier(n_estimators=167, random_state = 90, n_jobs=-1)
GS = GridSearchCV(model_rf, param_grid, cv=10)
GS.fit(data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y.values.ravel())
print(GS.best_params_)
print(GS.best_score_)

{'max_depth': 19}
0.9983038939425459


In [ ]:
#{'max_depth': 19}
# 0.9983038939425459

In [60]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=19,n_jobs=-1)

# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(max_depth=19, n_estimators=167, n_jobs=-1,
                       random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_recall  test_precision  test_roc_auc  \
avg  4.371882    0.248821     0.998545        0.997822      0.999987   
std  0.730694    0.024964     0.000833        0.001414      0.000009   

     test_accuracy   test_f1  
avg       0.998260  0.998183  
std       0.000645  0.000673  


In [61]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_1.csv',index=None)

RandomForestClassifier(max_depth=19, n_estimators=167, n_jobs=-1,
                       random_state=90) 

sum: 10022


#### Round #4 - max_features

In [64]:
param_grid = {'max_features':np.arange(1,30,1)}
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=19,n_jobs=-1)
GS = GridSearchCV(model_rf, param_grid, cv=10)
GS.fit(data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y.values.ravel())
print(GS.best_params_)
print(GS.best_score_)

{'max_features': 16}
0.9984778637214667


In [ ]:
# {'max_features': 16}
# 0.9984778637214667

In [66]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=19, max_features=16, n_jobs=-1)

# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(max_depth=19, max_features=16, n_estimators=167,
                       n_jobs=-1, random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_recall  test_precision  test_roc_auc  \
avg  6.147662    0.214846     0.998545        0.997550      0.999988   
std  1.296317    0.026625     0.000833        0.001345      0.000006   

     test_accuracy   test_f1  
avg       0.998130  0.998047  
std       0.000552  0.000576  


In [67]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_4.csv',index=None)

RandomForestClassifier(max_depth=19, max_features=16, n_estimators=167,
                       n_jobs=-1, random_state=90) 

sum: 11687


#### Round #5 - min_samples_leaf

In [69]:
param_grid = {'min_samples_leaf':np.arange(1,10,1)}
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=19,max_features=16,n_jobs=-1)
GS = GridSearchCV(model_rf, param_grid, cv=10)
GS.fit(data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y.values.ravel())
print(GS.best_params_)
print(GS.best_score_)

{'min_samples_leaf': 1}
0.9984778637214667


In [ ]:
# {'min_samples_leaf': 1}
# 0.9984778637214667

In [70]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=19,max_features=16,min_samples_leaf=1, n_jobs=-1)


# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(max_depth=19, max_features=16, n_estimators=167,
                       n_jobs=-1, random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_recall  test_precision  test_roc_auc  \
avg  6.306694    0.217128     0.998545        0.997550      0.999988   
std  1.331241    0.033789     0.000833        0.001345      0.000006   

     test_accuracy   test_f1  
avg       0.998130  0.998047  
std       0.000552  0.000576  


In [72]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_5.csv',index=None)

RandomForestClassifier(max_depth=19, max_features=16, n_estimators=167,
                       n_jobs=-1, random_state=90) 

sum: 11687


#### Round #6 - min_samples_split

In [77]:
param_grid = {'min_samples_split':np.arange(1,7,1)}
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=19,max_features=16,min_samples_leaf=1, n_jobs=-1)
GS = GridSearchCV(model_rf, param_grid, cv=10)
GS.fit(data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y.values.ravel())
print(GS.best_params_)
print(GS.best_score_)

/Users/yankun/yankun/Python/anaconda3/envs/sklearn-env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/yankun/yankun/Python/anaconda3/envs/sklearn-env/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 431, in _process_worker
    r = call_item()
  File "/Users/yankun/yankun/Python/anaconda3/envs/sklearn-env/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 285, in __call__
    return self.fn(*self.args, **self

{'min_samples_split': 2}
0.9984778637214667


In [ ]:
# {'min_samples_split': 2}
# 0.9984778637214667

In [75]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, min_samples_split=2, n_estimators=167,max_depth=19,max_features=16,min_samples_leaf=1, n_jobs=-1)


# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(max_depth=19, max_features=16, n_estimators=167,
                       n_jobs=-1, random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_recall  test_precision  test_roc_auc  \
avg  7.617160    0.302484     0.998545        0.997550      0.999988   
std  1.470719    0.117559     0.000833        0.001345      0.000006   

     test_accuracy   test_f1  
avg       0.998130  0.998047  
std       0.000552  0.000576  


In [76]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_6.csv',index=None)

RandomForestClassifier(max_depth=19, max_features=16, n_estimators=167,
                       n_jobs=-1, random_state=90) 

sum: 11687


#### Round #7 - criterion

In [78]:
param_grid = {'criterion':['gini','entropy']}
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=19,max_features=16,min_samples_leaf=1, min_samples_split=2, n_jobs=-1)
GS = GridSearchCV(model_rf, param_grid, cv=10)
GS.fit(data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y.values.ravel())
print(GS.best_params_)
print(GS.best_score_)

{'criterion': 'gini'}
0.9984778637214667


In [ ]:
# {'criterion': 'gini'}
# 0.9984778637214667

In [79]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=19,max_features=16,min_samples_leaf=1, min_samples_split=2, criterion='gini', n_jobs=-1)


# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(max_depth=19, max_features=16, n_estimators=167,
                       n_jobs=-1, random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_recall  test_precision  test_roc_auc  \
avg  9.644214    0.253858     0.998545        0.997550      0.999988   
std  3.092279    0.024724     0.000833        0.001345      0.000006   

     test_accuracy   test_f1  
avg       0.998130  0.998047  
std       0.000552  0.000576  


In [80]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_7.csv',index=None)

RandomForestClassifier(max_depth=19, max_features=16, n_estimators=167,
                       n_jobs=-1, random_state=90) 

sum: 11687


# Conclusion
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=19,max_features=16,min_samples_leaf=1, min_samples_split=2, criterion='gini', n_jobs=-1)

sum: 11687
Kaggle: 0.89703

It seems that it is overfitting and need to prune.


# Step 1: prune leaf  - min_samples_leaf=15

#### Round #1 - It seems that it is overfitting and need to prune.
min_samples_leaf=12

In [81]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=19,max_features=16,min_samples_leaf=12, min_samples_split=2, criterion='gini', n_jobs=-1)


# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(max_depth=19, max_features=16, min_samples_leaf=12,
                       n_estimators=167, n_jobs=-1, random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_recall  test_precision  test_roc_auc  \
avg  7.150638    0.248888     0.998818        0.994667      0.999950   
std  1.544064    0.014645     0.001079        0.002827      0.000029   

     test_accuracy   test_f1  
avg       0.996869  0.996736  
std       0.001386  0.001441  


In [82]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_pruning_1.csv',index=None)

RandomForestClassifier(max_depth=19, max_features=16, min_samples_leaf=12,
                       n_estimators=167, n_jobs=-1, random_state=90) 

sum: 11795


#### Round #2 - It seems that it is overfitting and need to prune.
min_samples_leaf=13

In [85]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=19,max_features=16,min_samples_leaf=13, min_samples_split=2, criterion='gini', n_jobs=-1)


# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(max_depth=19, max_features=16, min_samples_leaf=13,
                       n_estimators=167, n_jobs=-1, random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_recall  test_precision  test_roc_auc  \
avg  6.554919    0.251890     0.998818        0.994306      0.999946   
std  1.408883    0.043569     0.001079        0.002818      0.000031   

     test_accuracy   test_f1  
avg       0.996695  0.996555  
std       0.001583  0.001646  


In [86]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_pruning_2.csv',index=None)

RandomForestClassifier(max_depth=19, max_features=16, min_samples_leaf=13,
                       n_estimators=167, n_jobs=-1, random_state=90) 

sum: 11921


#### Round #3 - It seems that it is overfitting and need to prune.
min_samples_leaf=14

In [87]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=19,max_features=16,min_samples_leaf=14, min_samples_split=2, criterion='gini', n_jobs=-1)


# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(max_depth=19, max_features=16, min_samples_leaf=14,
                       n_estimators=167, n_jobs=-1, random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_recall  test_precision  test_roc_auc  \
avg  6.335993    0.225615     0.998909        0.994129      0.999945   
std  1.373405    0.027147     0.001060        0.002958      0.000032   

     test_accuracy  test_f1  
avg       0.996651  0.99651  
std       0.001443  0.00150  


In [88]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_pruning_3.csv',index=None)

RandomForestClassifier(max_depth=19, max_features=16, min_samples_leaf=14,
                       n_estimators=167, n_jobs=-1, random_state=90) 

sum: 11939


#### Round #4 - It seems that it is overfitting and need to prune.
min_samples_leaf=15

In [89]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=19,max_features=16,min_samples_leaf=15, min_samples_split=2, criterion='gini', n_jobs=-1)


# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(max_depth=19, max_features=16, min_samples_leaf=15,
                       n_estimators=167, n_jobs=-1, random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_recall  test_precision  test_roc_auc  \
avg  7.034704    0.238017     0.998727        0.994035      0.999942   
std  1.717319    0.022558     0.001012        0.002612      0.000035   

     test_accuracy   test_f1  
avg       0.996521  0.996374  
std       0.001305  0.001357  


In [90]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_pruning_4.csv',index=None)

RandomForestClassifier(max_depth=19, max_features=16, min_samples_leaf=15,
                       n_estimators=167, n_jobs=-1, random_state=90) 

sum: 11858


# Step 2: prune depth - max_depth=16

#### Round #1 - It seems that it is overfitting and need to prune.
max_depth=16

In [97]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=16,max_features=16,min_samples_leaf=14, min_samples_split=2, criterion='gini', n_jobs=-1)


# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(max_depth=16, max_features=16, min_samples_leaf=14,
                       n_estimators=167, n_jobs=-1, random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_recall  test_precision  test_roc_auc  \
avg  6.431815    0.258841     0.998909        0.994218      0.999945   
std  1.287924    0.052618     0.001060        0.002846      0.000032   

     test_accuracy   test_f1  
avg       0.996695  0.996555  
std       0.001378  0.001432  


In [98]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_pruning_4_1.csv',index=None)

RandomForestClassifier(max_depth=16, max_features=16, min_samples_leaf=14,
                       n_estimators=167, n_jobs=-1, random_state=90) 

sum: 11950


#### Round #2 - It seems that it is overfitting and need to prune.
max_depth=13

In [102]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=13,max_features=16,min_samples_leaf=14, min_samples_split=2, criterion='gini', n_jobs=-1)


# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(max_depth=13, max_features=16, min_samples_leaf=14,
                       n_estimators=167, n_jobs=-1, random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_recall  test_precision  test_roc_auc  \
avg  7.224528    0.254352     0.998727        0.994126      0.999944   
std  1.591956    0.031294     0.001233        0.002611      0.000031   

     test_accuracy   test_f1  
avg       0.996564  0.996419  
std       0.001222  0.001270  


In [103]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_pruning_4_2.csv',index=None)

RandomForestClassifier(max_depth=13, max_features=16, min_samples_leaf=14,
                       n_estimators=167, n_jobs=-1, random_state=90) 

sum: 11891


In [104]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=19,max_features=16,min_samples_leaf=11, min_samples_split=2, criterion='gini', n_jobs=-1)


# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(max_depth=19, max_features=16, min_samples_leaf=11,
                       n_estimators=167, n_jobs=-1, random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_recall  test_precision  test_roc_auc  \
avg  7.605271    0.248586     0.998818        0.994846      0.999953   
std  1.910973    0.040540     0.001000        0.002544      0.000029   

     test_accuracy   test_f1  
avg       0.996956  0.996826  
std       0.001245  0.001295  


In [105]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_pruning_.csv',index=None)

RandomForestClassifier(max_depth=19, max_features=16, min_samples_leaf=11,
                       n_estimators=167, n_jobs=-1, random_state=90) 

sum: 12084


# Step 3: prune depth - max_features=16

#### Round #1 - It seems that it is overfitting and need to prune.
max_features=13

In [129]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, n_estimators=167,max_depth=16,max_features=13,min_samples_leaf=14, min_samples_split=2, criterion='gini', n_jobs=-1)


# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(max_depth=16, max_features=13, min_samples_leaf=14,
                       n_estimators=167, n_jobs=-1, random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_recall  test_precision  test_roc_auc  \
avg  6.267131    0.257075     0.998818        0.994486      0.999944   
std  1.013279    0.026597     0.001000        0.002492      0.000034   

     test_accuracy   test_f1  
avg       0.996782  0.996645  
std       0.001087  0.001129  


In [130]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_pruning_4_1_1.csv',index=None)

RandomForestClassifier(max_depth=16, max_features=13, min_samples_leaf=14,
                       n_estimators=167, n_jobs=-1, random_state=90) 

sum: 11804


# Step 4: prune depth - n_estimators=

#### Round #1 - It seems that it is overfitting and need to prune.
min_samples_split=10

In [182]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, n_estimators=175,max_depth=16,max_features=16,min_samples_leaf=14, min_samples_split=2, criterion='gini', n_jobs=-1)



# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(max_depth=16, max_features=16, min_samples_leaf=14,
                       n_estimators=175, n_jobs=-1, random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_recall  test_precision  test_roc_auc  \
avg  7.619987    0.254105      0.99900        0.994038      0.999945   
std  1.474687    0.016324      0.00111        0.002818      0.000032   

     test_accuracy  test_f1  
avg       0.996651  0.99651  
std       0.001443  0.00150  


In [183]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_pruning_4_1_0_175.csv',index=None)

RandomForestClassifier(max_depth=16, max_features=16, min_samples_leaf=14,
                       n_estimators=175, n_jobs=-1, random_state=90) 

sum: 11979


In [22]:
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^ train models: RF, GRB, MLP, DT ^^^^^^^^^^^^^^^^^^^^^^^^^^^
# repeat several times for parameters tuning
# author:           Kun Yan
# student number:   300259303
# data:             2021-10-03
# Python version:   3.9.7
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

mm = MinMaxScaler()
selector = VarianceThreshold(np.median(data_aftcat_aftif.var().values))
model_rf = RandomForestClassifier(random_state = 90, n_estimators=175,max_depth=11,max_features=20,min_samples_leaf=13, min_samples_split=5, criterion='gini', n_jobs=-1)



# -------------------- model: RF (repeat several times for parameters tuning)-----------------------
print(model_rf)
score_sf_rf,score_sf_re_rf = train_model(mm, selector, model_rf, data_aftcat_aftif, data_aftcat_aftif_y)
print("score_sf_re_rf: \n", pd.DataFrame(score_sf_re_rf))
RF = pd.DataFrame(score_sf_rf['test_f1'],columns = ["RF"])
#print(RF)

RandomForestClassifier(max_depth=11, max_features=20, min_samples_leaf=13,
                       min_samples_split=5, n_estimators=175, n_jobs=-1,
                       random_state=90) 

score_sf_re_rf: 
      fit_time  score_time  test_roc_auc  test_accuracy  test_recall  \
avg  8.125925    0.249331      0.999946       0.996564     0.998727   
std  1.633343    0.020041      0.000030       0.001311     0.000833   

     test_precision   test_f1  
avg        0.994124  0.996419  
std        0.002517  0.001363  


In [23]:
#  -----------------------------Prediction RF -------------------
pre_rf = pre_submission(model_rf,data_aftcat_aftif_aftmm_aftfs, data_aftcat_aftif_y, testdata_aftcat_aftmm_aftfs)
pre_rf.to_csv(r'./submission_rf_pruning_4_1_0_t3.csv',index=None)

RandomForestClassifier(max_depth=11, max_features=20, min_samples_leaf=13,
                       min_samples_split=5, n_estimators=175, n_jobs=-1,
                       random_state=90) 

sum: 12138
